# Predicting Diabetes Using BRFSS Data

In [1]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

C:\Users\shrus\AppData\Roaming\Python\Python312\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
pd.set_option("display.max_columns",850)
pd.set_option("display.max_rows",85)

##### Convert data from a SAS format to CSV format

In [3]:
#temp_df=pd.read_sas("LLCP2022.XPT")

In [4]:
#temp_df.to_csv('BRFSS_2022.csv', sep=",", index=False)

In [5]:
BRFSS_2022 = pd.read_csv('BRFSS_2022.csv')
BRFSS_2022

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,PVTRESD1,COLGHOUS,STATERE1,CELPHON1,LADULT1,COLGSEX1,NUMADULT,LANDSEX1,NUMMEN,NUMWOMEN,RESPSLCT,SAFETIME,CTELNUM1,CELLFON5,CADULT1,CELLSEX1,PVTRESD3,CCLGHOUS,CSTATE1,LANDLINE,HHADULT,SEXVAR,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINSR,PERSDOC3,MEDCOST1,CHECKUP1,EXERANY2,SLEPTIM1,LASTDEN4,RMVTETH4,CVDINFR4,CVDCRHD4,CVDSTRK3,ASTHMA3,ASTHNOW,CHCSCNC1,CHCOCNC1,CHCCOPD3,ADDEPEV3,CHCKDNY2,HAVARTH4,DIABETE4,DIABAGE4,MARITAL,EDUCA,RENTHOM1,NUMHHOL4,NUMPHON4,CPDEMO1C,VETERAN3,EMPLOY1,CHILDREN,INCOME3,PREGNANT,WEIGHT2,HEIGHT3,DEAF,BLIND,DECIDE,DIFFWALK,DIFFDRES,DIFFALON,HADMAM,HOWLONG,CERVSCRN,CRVCLCNC,CRVCLPAP,CRVCLHPV,HADHYST2,HADSIGM4,COLNSIGM,COLNTES1,SIGMTES1,LASTSIG4,COLNCNCR,VIRCOLO1,VCLNTES2,SMALSTOL,STOLTEST,STOOLDN2,BLDSTFIT,SDNATES1,SMOKE100,SMOKDAY2,USENOW3,ECIGNOW2,LCSFIRST,LCSLAST,LCSNUMCG,LCSCTSC1,LCSSCNCR,LCSCTWHN,ALCDAY4,AVEDRNK3,DRNK3GE5,MAXDRNKS,FLUSHOT7,FLSHTMY3,PNEUVAC4,TETANUS1,HIVTST7,HIVTSTD3,HIVRISK5,COVIDPOS,COVIDSMP,COVIDPRM,PDIABTS1,PREDIAB2,DIABTYPE,INSULIN1,CHKHEMO3,EYEEXAM1,DIABEYE1,DIABEDU1,FEETSORE,TOLDCFS,HAVECFS,WORKCFS,IMFVPLA3,HPVADVC4,HPVADSHT,SHINGLE2,COVIDVA1,COVACGET,COVIDNU1,COVIDINT,COVIDFS1,COVIDSE1,COPDCOGH,COPDFLEM,COPDBRTH,COPDBTST,COPDSMOK,CNCRDIFF,CNCRAGE,CNCRTYP2,CSRVTRT3,CSRVDOC1,CSRVSUM,CSRVRTRN,CSRVINST,CSRVINSR,CSRVDEIN,CSRVCLIN,CSRVPAIN,CSRVCTL2,PSATEST1,PSATIME1,PCPSARS2,PSASUGST,PCSTALK1,CIMEMLOS,CDHOUSE,CDASSIST,CDHELP,CDSOCIAL,CDDISCUS,CAREGIV1,CRGVREL4,CRGVLNG1,CRGVHRS1,CRGVPRB3,CRGVALZD,CRGVPER1,CRGVHOU1,CRGVEXPT,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT1,ACESWEAR,ACETOUCH,ACETTHEM,ACEHVSEX,ACEADSAF,ACEADNED,LSATISFY,EMTSUPRT,SDHISOLT,SDHEMPLY,FOODSTMP,SDHFOOD1,SDHBILLS,SDHUTILS,SDHTRNSP,SDHSTRE1,MARIJAN1,MARJSMOK,MARJEAT,MARJVAPE,MARJDAB,MARJOTHR,USEMRJN4,LASTSMK2,STOPSMK2,MENTCIGS,MENTECIG,HEATTBCO,ASBIALCH,ASBIDRNK,ASBIBING,ASBIADVC,ASBIRDUC,FIREARM5,GUNLOAD,LOADULK2,RCSGEND1,RCSXBRTH,RCSRLTN2,CASTHDX2,CASTHNO2,BIRTHSEX,SOMALE,SOFEMALE,TRNSGNDR,HADSEX,PFPPRVN4,TYPCNTR9,BRTHCNT4,WHEREGET,NOBCUSE8,BCPREFER,RRCLASS3,RRCOGNT2,RRTREAT,RRATWRK2,RRHCARE4,RRPHYSM2,QSTVER,QSTLANG,_METSTAT,_URBSTAT,MSCODE,_STSTR,_STRWT,_RAWRAKE,_WT2RAKE,_IMPRACE,_CHISPNC,_CRACE2,_CPRACE2,CAGEG,_CLLCPWT,_DUALUSE,_DUALCOR,_LLCPWT2,_LLCPWT,_RFHLTH,_PHYS14D,_MENT14D,_HLTHPLN,_HCVU652,_TOTINDA,_EXTETH3,_ALTETH3,_DENVST3,_MICHD,_LTASTH1,_CASTHM1,_ASTHMS1,_DRDXAR2,_PRACE2,_MRACE2,_HISPANC,_RACE1,_RACEG22,_RACEGR4,_RACEPR1,_SEX,_AGEG5YR,_AGE65YR,_AGE80,_AGE_G,HTIN4,HTM4,WTKG3,_BMI5,_BMI5CAT,_RFBMI5,_CHLDCNT,_EDUCAG,_INCOMG1,_RFMAM22,_MAM5023,_HADCOLN,_CLNSCP1,_HADSIGM,_SGMSCP1,_SGMS101,_RFBLDS5,_STOLDN1,_VIRCOL1,_SBONTI1,_CRCREC2,_SMOKER3,_RFSMOK3,_CURECI2,_YRSSMOK,_PACKDAY,_PACKYRS,_YRSQUIT,_SMOKGRP,_LCSREC,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1.0,1.0,b'02032022',b'02',b'03',b'2022',1100.0,b'2022000001',2.022000e+09,1.0,1.0,NaN,1.0,2.0,1.0,NaN,2.0,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,88.0,88.0,NaN,99.0,1.0,2.0,1.0,2.0,8.0,NaN,NaN,2.0,2.0,2.0,2.0,NaN,2.0,2.0,2.0,2.0,2.0,2.0,1.0,80.0,1.0,6.0,1.0,1.0,1.0,2.0,2.0,7.0,88.0,99.0,NaN,9999.0,9999.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,NaN,NaN,NaN,2.0,1.0,3.0,2.0,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,3.0,4.0,NaN,NaN,NaN,2.0,NaN,NaN,888.0,NaN,NaN,NaN,1.0,92021.0,2.0,3.0,2.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,5.0,2.0,2.0,5.0,2.0,2.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.0,1.0,1.0,2.0,11011.0,37.418416,2.0,74.836832,1.0,9.0,NaN,NaN,NaN,NaN,1.0,0.520383,813.918517,487.612985,1.0,1.0,1.0,9.0,

##### Create datasets with selected variables

The selected features from the BRFSS 2022 dataset are:

Response Variable / Dependent Variable:

-Diabetes
(Ever told) (you had) diabetes? (If ´Yes´ and respondent is female, ask ´Was this only when you were pregnant?´. If Respondent says pre-diabetes or borderline diabetes, use response code 4.) --> 
DIABETE4  


Independent Variables:

1. Demographics

Fourteen-level age category --> _AGEG5YR 

Calculated sex variable --> _SEX 

What is the highest grade or year of school you completed? --> EDUCAG 

Income categories --> _INCOMG1   

2. COVID

Has a doctor, nurse, or other health professional ever told you that you tested positive for COVID 19? --> COVIDPOS (1,3 is yes, 2 is no, 7,9, blank is nan)

3. BMI

Body Mass Index (BMI) --> _BMI5 

4. Smoking

Have you smoked at least 100 cigarettes in your entire life? [Note: 5 packs = 100 cigarettes] --> SMOKE100 

Four-level smoker status:  Everyday smoker, Someday smoker, Former smoker, Non-smoker  --> _SMOKER3 

5. Alcohol Consumption

Heavy drinkers (adult men having more than 14 drinks per week and adult women having more than 7 drinks per week) --> _RFDRHV8 

6. Other Chronic Health Conditions

(Ever told) (you had) a depressive disorder (including depression, major depression, dysthymia, or minor depression)? --> ADDEPEV3  

(Ever told) (you had) chronic obstructive pulmonary disease, C.O.P.D., emphysema or chronic bronchitis? --> CHCCOPD3  

Computed asthma status --> _ASTHMS1    

(Ever told) you had a stroke --> CVDSTRK3   

(Ever told) you had a heart attack, also called a myocardial infarction? --> CVDINFR4     

(Ever told) you had angina or coronary heart disease? --> CVDCRHD4               

Not including kidney stones, bladder infection or incontinence, were you ever told you had kidney disease?  --> CHCKDNY2  

Respondents who have had a doctor diagnose them as having some form of arthritis -->  _DRDXAR2

Not including teeth lost for injury or orthodontics, how many of your permanent teeth have been removed because of tooth decay or gum disease? --> RMVTETH4

7. Psyhical Activity

Adults who reported doing physical activity or exercise during the past 30 days other than their regular job --> TOTINDA

8. Health Care

Was there a time in the past 12 months when you needed to see a doctor but could not because of cost? --> MEDCOST1 

Do you have any kind of health care coverage, including health insurance, prepaid plans such as HMOs, or government plans such as Medicare, or Indian Health Service? --> _HLTHPLN

9. Health General and Mental Health

Would you say that in general your health is: --> GENHLTH  

Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good? --> MENTHLTH

Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good? --> PHYSHLTH    

Do you have serious difficulty walking or climbing stairs? --> DIFFWALK 


In [14]:
df = BRFSS_2022[['DIABETE4', 
                 '_BMI5', 
                 '_SMOKER3', '_RFDRHV8',
                 'ADDEPEV3', 'CHCCOPD3', '_ASTHMS1', 'CVDSTRK3', 'CVDINFR4', 'CVDCRHD4','CHCKDNY2', '_DRDXAR2','RMVTETH4',
                 '_TOTINDA',
                 'MEDCOST1', '_HLTHPLN',
                 'GENHLTH', 'MENTHLTH', 'PHYSHLTH', 'DIFFWALK',
                 '_AGEG5YR', '_SEX', '_EDUCAG', '_INCOMG1','COVIDPOS']].astype(float)
df

,DIABETE4,_BMI5,_SMOKER3,_RFDRHV8,ADDEPEV3,CHCCOPD3,_ASTHMS1,CVDSTRK3,CVDINFR4,CVDCRHD4,CHCKDNY2,_DRDXAR2,RMVTETH4,_TOTINDA,MEDCOST1,_HLTHPLN,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,_AGEG5YR,_SEX,_EDUCAG,_INCOMG1,COVIDPOS
0,1.0,NaN,4.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,9.0,2.0,88.0,88.0,2.0,13.0,2.0,4.0,9.0,2.0
1,3.0,2657.0,4.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,2.0,1.0,1.0,88.0,88.0,2.0,13.0,2.0,2.0,3.0,2.0
2,3.0,2561.0,4.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,2.0,1.0,2.0,3.0,2.0,2.0,8.0,2.0,4.0,6.0,1.0
3,3.0,2330.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,NaN,1.0,2.0,9.0,1.0,88.0,88.0,2.0,14.0,2.0,2.0,9.0,2.0
4,3.0,2177.0,4.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,2.0,1.0,4.0,88.0,2.0,2.0,5.0,2.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,3.0,2563.0,4.0,9.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,3.0,3.0,88.0,2.0,1.0,2.0,2.0,1.0,1.0
445128,3.0,2866.0,4.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,7.0,2.0,4.0,5.0,2.0
445129,3.0,1723.0,1.0,9.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,5.0,30.0,30.0,2.0,10.0,2.0,2.0,9.0,2.0
445130,3.0,3255.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,8.0,2.0,2.0,1.0,2.0,88.0,88.0,2.0,11.0,1.0,3.0,5.0,1.0


In [15]:
df.shape

(445132, 25)

##### Clean the data

In [16]:
df = df.dropna()  #see how to drop the rows just based on on particular column--> DIABETE4
df.shape

(365838, 25)

In [17]:
df.isnull().sum()

DIABETE4    0
_BMI5       0
_SMOKER3    0
_RFDRHV8    0
ADDEPEV3    0
CHCCOPD3    0
_ASTHMS1    0
CVDSTRK3    0
CVDINFR4    0
CVDCRHD4    0
CHCKDNY2    0
_DRDXAR2    0
RMVTETH4    0
_TOTINDA    0
MEDCOST1    0
_HLTHPLN    0
GENHLTH     0
MENTHLTH    0
PHYSHLTH    0
DIFFWALK    0
_AGEG5YR    0
_SEX        0
_EDUCAG     0
_INCOMG1    0
COVIDPOS    0
dtype: int64

In [18]:
df = df.drop_duplicates()
df.shape

(363477, 25)

In [19]:
df.nunique()

DIABETE4       6
_BMI5       3893
_SMOKER3       5
_RFDRHV8       3
ADDEPEV3       4
CHCCOPD3       4
_ASTHMS1       4
CVDSTRK3       4
CVDINFR4       4
CVDCRHD4       4
CHCKDNY2       4
_DRDXAR2       2
RMVTETH4       6
_TOTINDA       3
MEDCOST1       4
_HLTHPLN       3
GENHLTH        7
MENTHLTH      33
PHYSHLTH      33
DIFFWALK       4
_AGEG5YR      14
_SEX           2
_EDUCAG        5
_INCOMG1       8
COVIDPOS       5
dtype: int64

In [20]:
def Diabetes_Clean(df):
    '''Removes values that are of no use to the later predictions. 
       Organizing data so that:
         0 is for no diabetes or only during pregnancy, 
         1 is for pre-diabetes or borderline diabetes, 
         2 is for yes diabetes
    '''
    
    df.drop(df[(df["DIABETE4"] == 7) | (df["DIABETE4"] == 9)].index, inplace=True)
    df['DIABETE4'] = [0 if (x == 2 or x == 3) else (1 if x == 4 else 2) for x in df['DIABETE4']]
    return df

Diabetes_Clean(df)

,DIABETE4,_BMI5,_SMOKER3,_RFDRHV8,ADDEPEV3,CHCCOPD3,_ASTHMS1,CVDSTRK3,CVDINFR4,CVDCRHD4,CHCKDNY2,_DRDXAR2,RMVTETH4,_TOTINDA,MEDCOST1,_HLTHPLN,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,_AGEG5YR,_SEX,_EDUCAG,_INCOMG1,COVIDPOS
342,0,2799.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,2.0,88.0,4.0,2.0,10.0,2.0,3.0,6.0,2.0
343,2,3013.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,2.0,88.0,88.0,2.0,11.0,1.0,3.0,5.0,2.0
344,0,2645.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,4.0,88.0,30.0,2.0,11.0,1.0,4.0,5.0,2.0
345,0,3166.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,88.0,88.0,1.0,12.0,1.0,4.0,5.0,1.0
346,0,3132.0,4.0,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,4.0,88.0,5.0,1.0,13.0,2.0,3.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,0,2563.0,4.0,9.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,3.0,3.0,88.0,2.0,1.0,2.0,2.0,1.0,1.0
445128,0,2866.0,4.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,7.0,2.0,4.0,5.0,2.0
445129,0,1723.0,1.0,9.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,5.0,30.0,30.0,2.0,10.0,2.0,2.0,9.0,2.0
445130,0,3255.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,8.0,2.0,2.0,1.0,2.0,88.0,88.0,2.0,11.0,1.0,3.0,5.0,1.0


In [21]:
def BMI_Clean(df):
    '''Change the BMI value to the classic scale from 1 to 100 and convert the values into categories. Based on Codebook.
         - 1 is Underweight (BMI less then 18.5)
         - 2 is Normal Weight (BMI between 18.5–24.9)
         - 3 is Overweight (BMI between 25 - 29.9)
         - 4 is Obesity I degree (BMI between 30–34,9)
         - 5 is Obesity II degree (BMI between 35–39.9)
         - 6 is Obesity III degree (BMI 40 and more)
    '''

    df['_BMI5'] = df['_BMI5'].div(100).round(1)
    df.loc[df['_BMI5'] < 18.5, '_BMI5'] = 1
    df.loc[(df['_BMI5'] >= 18.5) & (df['_BMI5'] <= 24.9), '_BMI5'] = 2
    df.loc[(df['_BMI5'] >= 25) & (df['_BMI5'] <= 29.9), '_BMI5'] = 3
    df.loc[(df['_BMI5'] >= 30) & (df['_BMI5'] <= 34.9), '_BMI5'] = 4
    df.loc[(df['_BMI5'] >= 35) & (df['_BMI5'] <= 39.9), '_BMI5'] = 5
    df.loc[df['_BMI5'] >= 40, '_BMI5'] = 6
    return df

BMI_Clean(df)

,DIABETE4,_BMI5,_SMOKER3,_RFDRHV8,ADDEPEV3,CHCCOPD3,_ASTHMS1,CVDSTRK3,CVDINFR4,CVDCRHD4,CHCKDNY2,_DRDXAR2,RMVTETH4,_TOTINDA,MEDCOST1,_HLTHPLN,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,_AGEG5YR,_SEX,_EDUCAG,_INCOMG1,COVIDPOS
342,0,3.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,2.0,88.0,4.0,2.0,10.0,2.0,3.0,6.0,2.0
343,2,4.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,2.0,88.0,88.0,2.0,11.0,1.0,3.0,5.0,2.0
344,0,3.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,4.0,88.0,30.0,2.0,11.0,1.0,4.0,5.0,2.0
345,0,4.0,3.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,88.0,88.0,1.0,12.0,1.0,4.0,5.0,1.0
346,0,4.0,4.0,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,4.0,88.0,5.0,1.0,13.0,2.0,3.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,0,3.0,4.0,9.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,3.0,3.0,88.0,2.0,1.0,2.0,2.0,1.0,1.0
445128,0,3.0,4.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,7.0,2.0,4.0,5.0,2.0
445129,0,1.0,1.0,9.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,5.0,30.0,30.0,2.0,10.0,2.0,2.0,9.0,2.0
445130,0,4.0,4.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,8.0,2.0,2.0,1.0,2.0,88.0,88.0,2.0,11.0,1.0,3.0,5.0,1.0


In [25]:
def Smoke_Clean(df):
    '''Removes values that are of no use to the later predictions. 
       Organizing data so that:
         - 0 is the respondent does not smoke cigarettes
         - 1 is the respondent smokes cigarettes
         - 2 is former smoker
    '''
    
    df.drop(df[(df["_SMOKER3"] == 7) | (df["_SMOKER3"] == 9)].index, inplace=True)
    df['_SMOKER3'] = [1 if (x == 1 or x == 2) else (0 if x == 4 else 2) for x in df['_SMOKER3']]
    return df

Smoke_Clean(df)

,DIABETE4,_BMI5,_SMOKER3,_RFDRHV8,ADDEPEV3,CHCCOPD3,_ASTHMS1,CVDSTRK3,CVDINFR4,CVDCRHD4,CHCKDNY2,_DRDXAR2,RMVTETH4,_TOTINDA,MEDCOST1,_HLTHPLN,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,_AGEG5YR,_SEX,_EDUCAG,_INCOMG1,COVIDPOS
342,0,3.0,2,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,2.0,88.0,4.0,2.0,10.0,2.0,3.0,6.0,2.0
343,2,4.0,2,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,2.0,88.0,88.0,2.0,11.0,1.0,3.0,5.0,2.0
344,0,3.0,2,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,4.0,88.0,30.0,2.0,11.0,1.0,4.0,5.0,2.0
345,0,4.0,2,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,88.0,88.0,1.0,12.0,1.0,4.0,5.0,1.0
346,0,4.0,0,1.0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,4.0,88.0,5.0,1.0,13.0,2.0,3.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445127,0,3.0,0,9.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,3.0,3.0,88.0,2.0,1.0,2.0,2.0,1.0,1.0
445128,0,3.0,0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,7.0,2.0,4.0,5.0,2.0
445129,0,1.0,1,9.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,5.0,30.0,30.0,2.0,10.0,2.0,2.0,9.0,2.0
445130,0,4.0,0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,8.0,2.0,2.0,1.0,2.0,88.0,88.0,2.0,11.0,1.0,3.0,5.0,1.0


In [27]:
def Alcohol_Clean(df):
    '''Removes values that are of no use to the later predictions. 
       Organizing data so that:
         - 0 is the respondent does not use alcohol,
         - 1 is the respondent is using alcohol
    '''
    
    df.drop(df[(df["_RFDRHV8"] == 9)].index, inplace=True)
    df['_RFDRHV8'] = [0 if x == 1 else 1 for x in df['_RFDRHV8']]
    return df

Alcohol_Clean(df)

,DIABETE4,_BMI5,_SMOKER3,_RFDRHV8,ADDEPEV3,CHCCOPD3,_ASTHMS1,CVDSTRK3,CVDINFR4,CVDCRHD4,CHCKDNY2,_DRDXAR2,RMVTETH4,_TOTINDA,MEDCOST1,_HLTHPLN,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,_AGEG5YR,_SEX,_EDUCAG,_INCOMG1,COVIDPOS
342,0,3.0,2,0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,2.0,88.0,4.0,2.0,10.0,2.0,3.0,6.0,2.0
343,2,4.0,2,0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,2.0,88.0,88.0,2.0,11.0,1.0,3.0,5.0,2.0
344,0,3.0,2,0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,4.0,88.0,30.0,2.0,11.0,1.0,4.0,5.0,2.0
345,0,4.0,2,0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,88.0,88.0,1.0,12.0,1.0,4.0,5.0,1.0
346,0,4.0,0,0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,8.0,1.0,2.0,1.0,4.0,88.0,5.0,1.0,13.0,2.0,3.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445123,0,2.0,0,0,1.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,1.0,2.0,4.0,7.0,88.0,2.0,2.0,2.0,4.0,2.0,1.0
445124,2,3.0,0,0,2.0,2.0,3.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,3.0,15.0,88.0,2.0,10.0,1.0,4.0,6.0,1.0
445126,0,4.0,0,0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,3.0,88.0,88.0,2.0,3.0,1.0,4.0,5.0,1.0
445128,0,3.0,0,0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,8.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,7.0,2.0,4.0,5.0,2.0
